### Packages installing
If there is some package missing then it can be installed here.

In [1]:
import sys
!{sys.executable} -m pip install Pillow

You are using pip version 18.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


### Imports

In [2]:
from pathlib import Path
import json
from datetime import datetime
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D, Flatten
from tensorflow.keras.applications.resnet_v2 import ResNet152V2, preprocess_input
# from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import History, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard

### Loading tensorboard extension

In [3]:
%load_ext tensorboard

### Constants

In [4]:
MODEL_SAVING_INFO = Path("__file__").absolute().parent.joinpath("results")
MODEL_SAVING_INFO.mkdir(exist_ok=True)
DATA_PATH = Path("__file__").absolute().parent.parent.joinpath("data")
EPOCHS = 10
VERBOSE = 1
VALIDATION_STEPS = 10
MONITOR = 'val_loss'
INPUT_SHAPE= (224, 224, 3)

### ImageDataGenerator
Using it for producing data generators, while using some additional options we can gain more data in case there is not enough for proper training.

In [5]:
# 1. regular 
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input)

# 2. with data augmentation
# train_datagen = ImageDataGenerator(
#     featurewise_center=True,
#     featurewise_std_normalization=True,
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     horizontal_flip=True,
#     preprocessing_function=preprocess_input)


# 3. We will not use data augmentation for validation and testing, because we want to test it on real data
validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

### Data generators
According to keras documentation, it is easier to use `ImageDataGenerators` to load data in a very optimal way. Data that we use is prepared to be loaded from directories. 

In [6]:
train_generator = train_datagen.flow_from_directory(
        DATA_PATH.joinpath('train'),
        target_size=(224, 224),
        batch_size=32,
        color_mode='rgb',
        class_mode='binary',
)

validation_generator = validation_datagen.flow_from_directory(
        DATA_PATH.joinpath('validation'),
        target_size=(224, 224),
        batch_size=32,
        color_mode='rgb',
        class_mode='binary')

test_generator = validation_datagen.flow_from_directory(
        DATA_PATH.joinpath('test'),
        target_size=(224, 224),
        batch_size=32,
        color_mode='rgb',
        class_mode='binary')

Found 526 images belonging to 2 classes.
Found 176 images belonging to 2 classes.
Found 176 images belonging to 2 classes.


### Preparing model - transfer learning
**Transfer learning** - is a technique of training models using pre-trained models, where we can use weights which were saved after training on huge datasets like `imagenet`. Also, it is hightly recommended to add some additional layers or just one for your own data classification. 

For experiments, there are two pre-trained models imported: `MobileNetV2` and `ResNet152V2`. Each of them can be used while imported. The default input size for both models is `224x224`, because I will use weights from `imagenet`, where models were trained on images of that size. 

In [7]:
# base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=INPUT_SHAPE)
base_model = ResNet152V2(weights='imagenet', include_top=False, input_shape=INPUT_SHAPE)

Adding layers part.

In [8]:
output = base_model.output
# output= GlobalAveragePooling2D()(output)
output = Flatten()(output)
output = Dense(512, activation="relu")(output)
output = Dense(1, activation="sigmoid", name="classification_layer")(output)
model = Model(inputs=base_model.input, outputs=output)

# if we want to make first 20 layers not trainable
for layer in model.layers[:20]:
    layer.trainable=False

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
______________________________________________________________________________________________

In [9]:
model.compile(optimizer='Adam',loss='binary_crossentropy', metrics=['accuracy'])

### Model callbacks and training

In [10]:
# preparing directory for new experiment data
now = datetime.today()
TIMESTAMP = now.strftime("%Y%m%d_%H%M%S")
current_model_path = MODEL_SAVING_INFO.joinpath(TIMESTAMP)
current_model_path.mkdir(exist_ok=True)


# preparing callbacks
model_checkpoint = ModelCheckpoint(str(current_model_path.joinpath("model.h5")), monitor=MONITOR, verbose=VERBOSE, save_best_only=True, save_weights_only=False, mode='auto')
early_stopping = EarlyStopping(monitor=MONITOR, min_delta=0.0001, patience=20, verbose=1, mode='auto', baseline=None)
reduce_lr = ReduceLROnPlateau(monitor=MONITOR, factor=0.2, patience=5, min_lr=0.001)
tensorboard = TensorBoard(log_dir=str(current_model_path.joinpath("logs")), histogram_freq=0, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None, update_freq='epoch')
print("Path to tensorboard logs:", current_model_path.joinpath("logs"))

# training and validation of model
history = model.fit_generator(
        train_generator,
        steps_per_epoch=train_generator.n//train_generator.batch_size,
        epochs=EPOCHS,
        verbose=VERBOSE,
        validation_data=validation_generator,
        validation_steps=VALIDATION_STEPS,
        callbacks=[
            model_checkpoint,
            early_stopping,
            reduce_lr,
            tensorboard
        ]
        )

# saving training history
with open(current_model_path.joinpath("history.json"), "w") as file:
    json.dump(str(history.history), file)

Path to tensorboard logs: /Users/ewelina/work/projects/artworks_classification/data_classification/results/20191104_230846/logs
Epoch 1/10
15/16 [===========================>..] - ETA: 32s - loss: 7.4368 - accuracy: 0.4827 
Epoch 00001: val_loss improved from inf to 7.51966, saving model to /Users/ewelina/work/projects/artworks_classification/data_classification/results/20191104_230846/model.h5
16/16 [==============================] - 654s 41s/step - loss: 7.4511 - accuracy: 0.4838 - val_loss: 7.5197 - val_accuracy: 0.5066
Epoch 2/10
15/16 [===========================>..] - ETA: 33s - loss: 7.8181 - accuracy: 0.4957 
Epoch 00002: val_loss did not improve from 7.51966
16/16 [==============================] - 654s 41s/step - loss: 7.8395 - accuracy: 0.4939 - val_loss: 7.9053 - val_accuracy: 0.4803
Epoch 3/10
15/16 [===========================>..] - ETA: 30s - loss: 7.6666 - accuracy: 0.5087 
Epoch 00003: val_loss did not improve from 7.51966
16/16 [==============================] - 582s 

KeyboardInterrupt: 

### To use tensorboard: `%tensorboard --logdir path_generated_above`

In [ ]:
%tensorboard --logdir /Users/ewelina/work/projects/artworks_classification/data_classification/results/20191104_192327/logs

### Testing model
After training and validating model, if it is working properly it should be tested on test data.

In [ ]:
history = model.evaluate_generator(
    test_generator,
    callbacks=[
            model_checkpoint,
            early_stopping,
            reduce_lr,
            tensorboard
        ],
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False,
    verbose=1
)

test_accuracy = history[1]
test_loss = history[0]

In [ ]:
print(f"Your model gained {test_accuracy} test accuracy value and {test_loss} test loss value.")